In [88]:
import yfinance as yf 
import pandas as pd 
import numpy as np 
import datetime as dt

# Loading in the values from YFinance data, which was aggregated monthly, then calculating the rolling mean of the volume

In [103]:

df = pd.read_csv("YFinance_data_cleaned.csv")
df['date'] = pd.to_datetime(df['yyyy'].astype(str) + df['mm'].astype(str), format='%Y%m')
df.sort_values(by=['tic', 'date'], inplace=True)
results = pd.DataFrame()
for ticker, group in df.groupby('tic'):
    rolling_window = group.rolling('365D', on='date')
    group['rolling_mean'] = rolling_window['Volume'].mean()

    results = pd.concat([results, group])

results['1YearVolumeRatio'] = results['Volume']/results['rolling_mean']

         Unnamed: 0  yyyy  mm       Open       High        Low      Close  \
1222632     1222632  2000   1  48.918097  49.765290  47.004649  48.319027   
1222633     1222633  2000   2  60.224427  63.608727  58.952521  61.807046   
1222634     1222634  2000   3  87.360826  91.856767  82.689944  87.663568   
1222635     1222635  2000   4  67.213030  70.281323  63.478842  66.965967   
1222636     1222636  2000   5  56.936046  59.407108  54.286156  56.341141   
...             ...   ...  ..        ...        ...        ...        ...   
1595585     1595585  2023   1   0.183640   0.183640   0.183640   0.183640   
1595586     1595586  2023   2   0.219100   0.219100   0.219100   0.219100   
1595587     1595587  2023   3   0.182887   0.182887   0.182887   0.182887   
1595588     1595588  2023   4   0.190000   0.190000   0.190000   0.190000   
1595589     1595589  2023   5   0.159500   0.159500   0.159500   0.159500   

         Adj Close        Volume    tic       date  rolling_mean  
1222632 

# Calculating RSI using Yfinance Data

In [179]:
import pandas as pd


df = pd.read_csv("YFinance_data_cleaned.csv")
df['date'] = pd.to_datetime(df['yyyy'].astype(str) + df['mm'].astype(str), format='%Y%m')
df.sort_values(by=['tic', 'date'], inplace=True)
def calculate_monthly_rsi(df):
    df['diff'] = df['Adj Close'].diff(1)
    df['gain'] = df['diff'].clip(lower=0).round(2).fillna(0)
    df['loss'] = df['diff'].clip(upper=0).abs().round(2).fillna(0)
    window_length = 12
    df['avg_gain'] = df['gain'].rolling(window=window_length, min_periods=1).mean()[:window_length+1]
    df['avg_loss'] = df['loss'].rolling(window=window_length, min_periods=1).mean()[:window_length+1]
    for i, row in enumerate(df['avg_gain'].iloc[window_length+1:]):
        df['avg_gain'].iloc[i + window_length + 1] =(df['avg_gain'].iloc[i + window_length] *(window_length - 1) +df['gain'].iloc[i + window_length + 1])/ window_length
    for i, row in enumerate(df['avg_loss'].iloc[window_length+1:]):
        df['avg_loss'].iloc[i + window_length + 1] =(df['avg_loss'].iloc[i + window_length] * (window_length - 1) + df['loss'].iloc[i + window_length + 1])/window_length
        df['rs'] = df['avg_gain'] / df['avg_loss']
        df['rsi'] = 100 - (100 / (1.0 + df['rs']))
    return df

rsi_data = df.groupby('tic').apply(calculate_monthly_rsi)

print(rsi_data)


               Unnamed: 0  yyyy  mm       Open       High        Low  \
tic                                                                    
A     1222632     1222632  2000   1  48.918097  49.765290  47.004649   
      1222633     1222633  2000   2  60.224427  63.608727  58.952521   
      1222634     1222634  2000   3  87.360826  91.856767  82.689944   
      1222635     1222635  2000   4  67.213030  70.281323  63.478842   
      1222636     1222636  2000   5  56.936046  59.407108  54.286156   
...                   ...   ...  ..        ...        ...        ...   
ZZZOF 1595585     1595585  2023   1   0.183640   0.183640   0.183640   
      1595586     1595586  2023   2   0.219100   0.219100   0.219100   
      1595587     1595587  2023   3   0.182887   0.182887   0.182887   
      1595588     1595588  2023   4   0.190000   0.190000   0.190000   
      1595589     1595589  2023   5   0.159500   0.159500   0.159500   

                   Close  Adj Close        Volume    tic       

In [183]:
rsi_data1 = rsi_data.drop(columns=['tic'])
rsi_data= rsi_data1.reset_index().dropna()
#Here only the first column is dropped cos you cant have RSI on the first data
rsi_data.to_pickle("rsi_data.pkl")

In [192]:
rsi_data = rsi_data.rename(columns={'yyyy': 'year', 'mm':'month'})
cri = pd.read_pickle("cri_compustat_merged_v5.pkl")
cri1 = cri.merge(rsi_data, on=['year','month','tic'], how='inner')
cri1.to_pickle("cri_yfinance_RSI.pkl")


Stock Market Indicators:
Stock volatility.
Beta (compared to a global market index like MSCI World Index).
Trading volume.
Stock price momentum.


these features im getting since our data is monthly then ill be taking monthly averages right or do we want High & Low & mean also

In [17]:
def download_data(ticker, _start =dt.date(2000,1,1), _end = dt.date(2023,5,31)):
    return yf.download(ticker, start=_start, end = _end)
    

In [ ]:
company_PV_data = pd.DataFrame()
unique_Tickers = compustat_data_ALL['tic'].unique()
for i in unique_Tickers:
    try:
        temp_df = download_data(i) 
        temp_df.reset_index(inplace=True)
        temp_df['yyyy'] = temp_df.Date.apply(lambda x: x.year)
        temp_df['mm'] = temp_df.Date.apply(lambda x: x.month)
        temp_df.drop(['Date'],axis=1, inplace=True)
        grouped = temp_df.groupby(by = ['yyyy','mm']).mean()
        grouped.reset_index(inplace=True)
        grouped['tic'] = i
        company_PV_data = pd.concat([company_PV_data, grouped])
    except Exception as e:
        print(e)
